# CBTH API Demonstration

This Jupyter Notebook will demonstrate the use of the CB ThreatHunter APIs through both using raw APIs and also using the `cbapi` Python module.

**NOTE** the support for ThreatHunter in `cbapi` is still under development and the interfaces are subject
to change.

## Prerequisites

There are two prerequisites for using this code: first, you need credentials to log into the API for your
Cb PSC organization; and second, you need the `cbapi` bindings to use this Python code directly. If you
want to use another language, or to call the REST API endpoints manually, you won't need to install `cbapi`.

### API Credentials

The first step is to create connectors in your Cb PSC organization. Log into the console and follow the
instructions at https://developer.carbonblack.com/reference/cb-defense/authentication/ to create an`API` type connector

Once you have your connector, you'll need the following information:

1. URL endpoint (e.g. `defense-prod05.conferdeploy.net`) for the APIs. This is the same URL you would use for the PSC Web UI
2. Connector ID and API key for the API connector
4. "Org key" - this is a unique identifier for your org and is displayed on the top of API Keys page

### Install cbapi

The second step is only if you want to run this code directly. This python script uses the `cbapi`
module. The support for ThreatHunter in `cbapi` is being actively developed in a fork available from
https://github.com/trailofbits/cbapi-python/tree/tob-cbth. To run this code as-is, you need to `git clone`
that repository, change into the `tob-cbth` branch, and install `cbapi` in a virtualenv.

`cbapi` uses credential file to read the API secret keys. Whenever you write scripts to interact with the
Cb APIs (or any API for that matter) you should **always** keep your API secret keys separate from your script.
If your script is ever exposed, either intentionally (by sharing it), or accidentally, then your API token
could be compromised if it were embedded inside your script.

To learn more about credential files and `cbapi`, see the docs at https://cbapi.readthedocs.io/en/latest/#api-credentials.

## Documentation

More information on configuring `cbapi`:
https://cbapi.readthedocs.io/en/latest/installation.html

Documentation for the ThreatHunter APIs is now available on the Developer Network website at: https://developer.carbonblack.com/reference/cb-threathunter/


## Feeds and Watchlists Data Model
Feeds and watchlist data model diagram shows relationships between entities
![title](model.png)
1. **Report** is primary entity. It is container of IOCs and can be combined into customizable watchlists
2. Report can have one or more **IOCs** that fall into two basic categories:
    1. Query IOCs - iocs based on Solr queries that are executed on incoming data on regular schedule (every 5 mins)
    2. Ingress IOCs - iocs based on single-field match that are executed as data is coming into the system
    
    You can create reports directly, either through UI or API. Note that Ingress IOC-based reports can only be created through the API today.
3. **Feed** is a container that allows packaging reports for publishing purpose. There are public feeds (managed by CB) or you can create your private feed as well (visible only to your organization). Feed reports get tagged with special feed_id **classifier** which allow you to reference them in the reports
4. **Watchlist** is collection of reports that will be executed against the incoming data. It  contains reference and not a copy of the reports. This means that, if report changes, all watchlists using it will pick up the change automatically. There are two ways you can create a watchlist:
    1. By including list of specific report Ids
    2. By including reference to a set of feeds. In this case, list of reports will be dynamic, and depend on publisher of a given feed
    Watchlist can be configured to **Tag** matching documents, **Alert** on them, or both
4. **Exclusions** can be set on either report or IOC level. Exclusions allow you to ignore specific reports or IOCs within a feed. This is specifically useful when subscribing to public feeds that you cannot control


## Imports

First things first, we import our `cbapi` module.

We can also enable debug logging, which will provide an output of the underlying REST API calls that 
are made to the backend.

Finally, we instantiate CbThreatHunterAPI using profile "devday" that we have configured outside of this Notbook.

In [23]:
# Import ThreatHunter and Defense modules.
# - the need to import Defense and use a separate API key will go away when the migration of platform
#   APIs to the Custom Connector type is complete.

from cbapi.psc.threathunter import *

# pretty printer for making the output more readable
import pprint
import time

# Import logging module to see the requests to the REST API

# import logging
# logging.basicConfig()
# logging.getLogger("cbapi").setLevel(logging.DEBUG)

th = CbThreatHunterAPI(profile="devday")    # This is the 'name' of your credential in the credential file
orgkey="7YZFGDDN"  # This is our orgkey. Please update with orgkey of the actual org you will manage

## Watchlists And Feeds

### Create new query report
First, let's create a new query-based report. It will be a single-IOC report that will detect notepads creating child processes using Solr query.

This created our private report. Now we can try to get it back from the watchlistmgr to make sure it was really added to the store. Note that you need to provide monothonically increasing timestamp every time you create or update the report, so best is to use current time.

In [25]:
ret = th.get_object(f"/threathunter/watchlistmgr/v3/orgs/{orgkey}/reports/{query_report_id}")
pprint.pprint(ret)

{'description': 'Detected an instance of notepad.exe creation one or more '
                'child processes',
 'id': 'WORCC7gpRqC2LLBqbgup4A',
 'iocs': None,
 'iocs_v2': [{'field': None,
              'id': 'query_notepad_ioc',
              'link': 'https://devday2019.carbonblack.com/notepadReport',
              'match_type': 'query',
              'values': ['process_name:notepad.exe childproc_count:[1 TO *]']}],
 'link': 'https://devday2019.carbonblack.com/notepadReport',
 'severity': 8,
 'tags': None,
 'timestamp': 1559586456,
 'title': 'Notepad spawning processes',
 'visibility': None}


In [ ]:
## Create Watchlist
Now that we have the report, let's create a watchlist for this report. Watchlist will execute this report on data as it is coming in through the system

Let's make sure that the new watchlist is in the system

In [27]:
ret = th.get_object(f"/threathunter/watchlistmgr/v3/orgs/{orgkey}/watchlists/{watchlist_id}")
pprint.pprint(ret)

{'alerts_enabled': False,
 'classifier': None,
 'create_timestamp': 1559586561,
 'description': 'Pretty cool',
 'id': 'sE6Hg7LRvqhy09kzxdxHw',
 'last_update_timestamp': 1559586561,
 'name': 'DevDay Test Watchlist',
 'report_ids': ['WORCC7gpRqC2LLBqbgup4A'],
 'tags_enabled': True}


## Add Ingress-based Report
Now let's create another report. This one will be ingress-based IOC report. Note that these types of reports cannot be created through the UI, so API is our only choice.

In [28]:
ret = th.post_object(f"/threathunter/watchlistmgr/v3/orgs/{orgkey}/reports", 
                   {
                       "timestamp": int(time.time()),
                       "link": "https://devday2019.carbonblack.com/notepadIOCReport",
                       "title": "Misspelled notepad",
                       "description": "Detected an instance of suspicious notepad.exe",
                       "severity": 7,
                       "iocs_v2": [
                           {
                               "id": "query_notepad_ioc2",
                               "match_type": "regex",
                               "field": "process_name",
                               "values": [".+/notep@d.exe"],
                               "link": "https://devday2019.carbonblack.com/notepadIOCReport"
                           },
                           {
                               "id": "query_notepad_ioc3",
                               "match_type": "equality",
                               "field": "process_hash",
                               "values": [
                                    "cc5a14ff026ee593d7d25f213715b73833e6b9cf71091317121a009d5ad7fc36",
                                    "28a8d6b8a0cdcb25d098e403cc8b6dcb855cb591f0b54c2e3363b5c580d92b28",
                                    "facbc2cb089668197ca3968a3433b6f4826430c13f7d1c75b44667307c67dfe3",
                                    "e714a5147335245c386b105bb7494a8b190b6a737ba28f029561efe48105cd11",
                                    "d9442f97b93e30eda70d26f1e8664bd6ab12360d9daaec64eb975356cb4f3371",
                                    "18cfbae517b4c469014d753e3fe169408f40b70ba6f6ca8ef47106d3ac001577",
                                    "3e4efa728f170f50d53028e1440ca819c4b27f20c45d87f920c377b8f02ed32b",
                                    "2d17eecb9591a489a85c473573bd30b61bc7f77eba43ee7da8b9a1289e325180",
                                    "2ff92874437b2f48b39a426e364b27bf24a6aa519b44f9128e624bd727173cb8",
                                    "ae147e750d0f4415f1285727301627b71e0e1c11ea048fdd0e7d5e0355f4ece6",
                                    "230c04fac8798b3f48d9ea8f4b1cba2896201887d0f9a1ad4836cf8e8e6027ce",
                                    "fdbdb71a3049161ea5c070922305bb97a437db0b54002a87672f121d4ad8b01d",
                                    "691de01cf755a509eb92aa759634719279b641b7710d0f91a49823a42947583c",
                                    "f2fd2beaebaa7f2a6f00c2118ce653fe0f853a6a643ab804f5b8bf7695c9c72b",
                                    "b0595609193e4c200960dcda6e13ccd9f51d34702342f8eb8ab95737ba28c2c0",
                                    "0aee74d7daf5918f0d6c46ba8d0f116459a744c09a2eba43755489aa2594af2f",
                                    "b1adc3b49f3f75c3d0b7b9ee080dc7ba40872e601e2ad87a3f25f40abfb8c658",
                                    "b20cf39284f185a31f81640c7bdc29fe73385b54f5a5d3cdcbb93b8f949c631c",
                                    "995309d4cdd9bdf79cbd207f6f5056bb9c8ba6e4ae8eab3d337fcecc049fadce",
                                    "00a66bdce043d6d0c8117990999fafe0d5e65f67b08d1d676816248092cf5798",
                                    "b8681a43b70dca967e8b82be5dcba7e68f72f00c98f20aa2b4d74fa86f1ed4bd",
                                    "27509bb585ef43cd9cb3a7894ade26e4240f187d9c979d21a0025c83fd86edfa",
                                    "a0141a44c805d9378d94a157a440a2fb9b1f56f1bc49a8dab9ebc53359b6f3d9",
                                    "3a21802fb70556dba8955d61ce2af7b060aa5f4e89d04996168b402c752be7b0",
                                    "082b148fafe2f1f143c98536139b923be8e1cc3f8f5f25dd4635f51042284417",
                                    "9790efd4884010129ce18e459e655d07e0a985f0de377c125b2f8f796a15eb4a",
                                    "cffbb27f76f67290eb483e62a97098c77d5aa01884dcf33b5a177924df3a96f8",
                                    "fd09b105f61c52dcf668176a417e3d0b4b1323cb43d7891c296deb41735916d4",
                                    "52a56e43eea9b680fc9e047b4cd1edaadb51e58868bc2466f8cb60274797844f",
                                    "b65676c32f888e6256932983e89d2bb8694ce6977cd6feacec0879820e010a1d",
                                    "2fe0eac1aaf4b02f8902bad62346e2a50ec5a322f6d7656007011c0df70b011f",
                                    "4d34def9f0d5b3faf09e506c1e59ad5eb9ab2b8f5199bc7a67e6b111e756f2a3",
                                    "7ae29c70af5c4e46de3b1218ede8cded4023c1e699a04a137396270ea0ce270e",
                                    "09503fc6344386583bce1385438cb1a5443d3e08dc28d1c2f2c77fce8d75ce78",
                                    "8ba53dc3ecdf79f12bb47c986bd4b70dfe2e8ec3842ed632ad6cf228a360d2fa",
                                    "d7a2b983981c8077b1cf87e648136c47c547626b7aa21b7eaff48d6360ff375f",
                                    "efa42b76c35ac574c1b4b9770b5aefef2a6315e235f1e05f8b330d41ab76fcc0"
                                    ],
                               "link": "https://devday2019.carbonblack.com/notepadIOCReport2"
                           }
                       ]
                   }
                  )
pprint.pprint(ret.json())
ioc_report_id = ret.json()["id"]

{'description': 'Detected an instance of suspicious notepad.exe',
 'id': 'cqtxuHzRYeAlgn80AdFUA',
 'iocs': None,
 'iocs_v2': [{'field': 'process_name',
              'id': 'query_notepad_ioc2',
              'link': 'https://devday2019.carbonblack.com/notepadIOCReport',
              'match_type': 'regex',
              'values': ['.+/notep@d.exe']},
             {'field': 'process_hash',
              'id': 'query_notepad_ioc3',
              'link': 'https://devday2019.carbonblack.com/notepadIOCReport2',
              'match_type': 'equality',
              'values': ['cc5a14ff026ee593d7d25f213715b73833e6b9cf71091317121a009d5ad7fc36',
                         '28a8d6b8a0cdcb25d098e403cc8b6dcb855cb591f0b54c2e3363b5c580d92b28',
                         'facbc2cb089668197ca3968a3433b6f4826430c13f7d1c75b44667307c67dfe3',
                         'e714a5147335245c386b105bb7494a8b190b6a737ba28f029561efe48105cd11',
                         'd9442f97b93e30eda70d26f1e8664bd6ab12360d9daaec64eb

## Update watchlist
Now let's add that report to our existing watchlist, and also enable alerting on that watchlist

In [29]:
ret = th.put_object(f"/threathunter/watchlistmgr/v3/orgs/{orgkey}/watchlists/{watchlist_id}", {
    "name": "DevDay Test Watchlist",
    "description": "Even cooler",
    "id": watchlist_id,
    "alerts_enabled": True,
    "tags_enabled": True,
    "report_ids": [query_report_id, ioc_report_id]}
)
pprint.pprint(ret.json())


{'alerts_enabled': True,
 'classifier': None,
 'create_timestamp': 1559586561,
 'description': 'Even cooler',
 'id': 'sE6Hg7LRvqhy09kzxdxHw',
 'last_update_timestamp': 1559586796,
 'name': 'DevDay Test Watchlist',
 'report_ids': ['cqtxuHzRYeAlgn80AdFUA', 'WORCC7gpRqC2LLBqbgup4A'],
 'tags_enabled': True}


## Introducing Feeds
Ok, now let's say we decide to share our malware research with team members. Also, we might want them to be able to subscribe to our research and get updates automatically.
This is when feeds come to play.
Carbon Black offers large number of community feeds you can subscribe to. Now, you can create your own.
Note that, when creating feed, you need to put report definitions in place - you cannot reference existing reports. Also, you will need to provide report id for each, that will be unique within the feed

In [30]:
ret = th.post_object("/threathunter/feedmgr/v1/feed", {
    "feedinfo":{
        "name": "Notepad attacks", 
        "provider_url": "https://devday2019.carbonblack.com/notepadFeed",
        "summary": "Various attacks using notepad",
        "category": "DevDay demo"
    }, 
    "reports":[{"timestamp": int(time.time()),
    "id": "report1",
    "link": "https://devday2019.carbonblack.com/notepadIOCReport",
      "title": "Misspelled notepad",
      "description": "Detected an instance of suspicious notepad.exe",
      "severity": 7,
      "iocs_v2": [
          {
              "id": "query_notepad_ioc2",
              "match_type": "regex",
              "field": "process_name",
              "values": [".+/notep@d.exe"],
              "link": "https://devday2019.carbonblack.com/notepadIOCReport"
          }
      ]
    },                    
   {"timestamp": int(time.time()),
    "id": "report2",
    "link": "https://devday2019.carbonblack.com/notepadReport",
      "title": "Notepad spawning processes",
      "description": "Detected an instance of notepad.exe creation one or more child processes",
      "severity": 8,
      "iocs_v2": [
          {
              "id": "query_notepad_ioc",
              "match_type": "query",
              "values": ["process_name:notepad.exe childproc_count:[1 TO *]"],
              "link": "https://devday2019.carbonblack.com/notepadReport"
          }
      ]
    }
]})
pprint.pprint(ret.json())
feed_id = ret.json()["id"]

{'access': 'private',
 'category': 'DevDay demo',
 'id': 'rjEcrqDUSyGwDRPt7A9Ocw',
 'name': 'Notepad attacks',
 'owner': '7YZFGDDN',
 'provider_url': 'https://devday2019.carbonblack.com/notepadFeed',
 'source_label': None,
 'summary': 'Various attacks using notepad'}


We can now modify our watchlist to use the feed instead of the reports

In [31]:
ret = th.put_object(f"/threathunter/watchlistmgr/v3/orgs/{orgkey}/watchlists/{watchlist_id}", 
                    {
                        "id": watchlist_id,
                        "name": "DevDay Test Watchlist",
                        "description": "Even cooler, now using feed",
                        "id": watchlist_id,
                        "alerts_enabled": True,
                        "tags_enabled": True,
                        'classifier': {'key': 'feed_id', 'value': feed_id}
                    }
)
pprint.pprint(ret.json())

{'alerts_enabled': True,
 'classifier': {'key': 'feed_id', 'value': 'rjEcrqDUSyGwDRPt7A9Ocw'},
 'create_timestamp': 1559586561,
 'description': 'Even cooler, now using feed',
 'id': 'sE6Hg7LRvqhy09kzxdxHw',
 'last_update_timestamp': 1559587085,
 'name': 'DevDay Test Watchlist',
 'report_ids': None,
 'tags_enabled': True}


In [32]:
ret = th.get_object(f"/threathunter/watchlistmgr/v3/orgs/{orgkey}/watchlists/{watchlist_id}")
pprint.pprint(ret)

{'alerts_enabled': True,
 'classifier': {'key': 'feed_id', 'value': 'rjEcrqDUSyGwDRPt7A9Ocw'},
 'create_timestamp': 1559586561,
 'description': 'Even cooler, now using feed',
 'id': 'sE6Hg7LRvqhy09kzxdxHw',
 'last_update_timestamp': 1559587085,
 'name': 'DevDay Test Watchlist',
 'report_ids': None,
 'tags_enabled': True}


## Updating Entire Feed
Main benefit now is that we can then modify our feed without changing watchlist. New and deleted reports will be updated in the watchlist automatically. Let's say that we weren't too happy about our detection by misspelled notepad name, but realized that notepads are almost never created by something else than explorer.exe process. 
We will now post new reports for these feeds. This will replace any old ones with the same ids and remove ones that are not in the new list.

**Important:** Timestamps are required to grow monothonically. If older or same timestamp is encountered form a given report, update to the report will be ignored by the backend. This is useful when you have a script that pushes your reports periodically (e.g. from the repo), and you don't want things that didn't change intentionally to propagate to all consumers.

In [33]:
ret = th.post_object(f"/threathunter/feedmgr/v2/orgs/{orgkey}/feeds/{feed_id}/reports", {
    "reports":[{"timestamp": int(time.time()),
                "id": "report2",
                "link": "https://devday2019.carbonblack.com/notepadReport",
                  "title": "Notepad spawning processes",
                  "description": "Detected an instance of notepad.exe creation one or more child processes",
                  "severity": 8,
                  "iocs_v2": [
                      {
                          "id": "query_notepad_ioc",
                          "match_type": "query",
                          "values": ["process_name:notepad.exe childproc_count:[1 TO *]"],
                          "link": "https://devday2019.carbonblack.com/notepadReport"
                      }
                  ]
                },
                {"timestamp": int(time.time()),
                 "id": "report3",
                 "link": "https://devday2019.carbonblack.com/notepadQueryReport2",
                 "title": "Unusually started notepad",
                 "description": "Detected an instance of notepad.exe started by something else than explorer.exe",
                 "severity": 7,
                 "iocs_v2": [
                     {
                         "id": "query_notepad_ioc3",
                         "match_type": "query",
                         "values": ["process_name:notepad.exe -parent_name:explorer.exe"],
                         "link": "https://devday2019.carbonblack.com/notepadQueryReport2"
                     }
                  ]
                }
]})
pprint.pprint(ret.json())


{'success': True}


Again, let's check how our new feed looks in the feedmgr:

In [34]:
ret = th.get_object(f"/threathunter/feedmgr/v2/orgs/{orgkey}/feeds/{feed_id}")
pprint.pprint(ret)

{'feedinfo': {'access': 'private',
              'category': 'DevDay demo',
              'id': 'rjEcrqDUSyGwDRPt7A9Ocw',
              'name': 'Notepad attacks',
              'owner': '7YZFGDDN',
              'provider_url': 'https://devday2019.carbonblack.com/notepadFeed',
              'source_label': None,
              'summary': 'Various attacks using notepad'},
 'reports': [{'description': 'Detected an instance of notepad.exe creation one '
                             'or more child processes',
              'id': 'report2',
              'iocs': None,
              'iocs_v2': [{'field': None,
                           'id': 'query_notepad_ioc',
                           'link': 'https://devday2019.carbonblack.com/notepadReport',
                           'match_type': 'query',
                           'values': ['process_name:notepad.exe '
                                      'childproc_count:[1 TO *]']}],
              'link': 'https://devday2019.carbonblack.com/notep

## Updating specific feed report
You can also update only a specific report within a feed

In [35]:
ret = th.put_object(f"/threathunter/feedmgr/v2/orgs/{orgkey}/feeds/{feed_id}/reports/report3", {
    "timestamp": int(time.time()),
    "link": "https://devday2019.carbonblack.com/notepadQueryReport2",
    "title": "Unusually started notepad",
    "description": "Detected an instance of notepad.exe started by something else than explorer.exe",
    "severity": 7,
    "iocs_v2": [
        {
            "id": "query_notepad_ioc",
            "match_type": "query",
            "values": ["process_name:notepad.exe -parent_name:explorer.exe -parent_name:cmd.exe"],
            "link": "https://devday2019.carbonblack.com/notepadQueryReport2"
        },
        {
            "id": "query_notepad_ioc4",
            "match_type": "query",
            "values": ["process_name:notepad.exe childproc_name:*"],
            "link": "https://devday2019.carbonblack.com/notepadQueryReport3"
        }
    ]
})
pprint.pprint(ret.json())


{'description': 'Detected an instance of notepad.exe started by something else '
                'than explorer.exe',
 'id': 'report3',
 'iocs': None,
 'iocs_v2': [{'field': None,
              'id': 'query_notepad_ioc',
              'link': 'https://devday2019.carbonblack.com/notepadQueryReport2',
              'match_type': 'query',
              'values': ['process_name:notepad.exe -parent_name:explorer.exe '
                         '-parent_name:cmd.exe']},
             {'field': None,
              'id': 'query_notepad_ioc4',
              'link': 'https://devday2019.carbonblack.com/notepadQueryReport3',
              'match_type': 'query',
              'values': ['process_name:notepad.exe childproc_name:*']}],
 'link': 'https://devday2019.carbonblack.com/notepadQueryReport2',
 'severity': 7,
 'tags': None,
 'timestamp': 1559587162,
 'title': 'Unusually started notepad',
 'visibility': None}


In [36]:
ret = th.get_object(f"/threathunter/feedmgr/v2/orgs/{orgkey}/feeds/{feed_id}")
pprint.pprint(ret)

{'feedinfo': {'access': 'private',
              'category': 'DevDay demo',
              'id': 'rjEcrqDUSyGwDRPt7A9Ocw',
              'name': 'Notepad attacks',
              'owner': '7YZFGDDN',
              'provider_url': 'https://devday2019.carbonblack.com/notepadFeed',
              'source_label': None,
              'summary': 'Various attacks using notepad'},
 'reports': [{'description': 'Detected an instance of notepad.exe creation one '
                             'or more child processes',
              'id': 'report2',
              'iocs': None,
              'iocs_v2': [{'field': None,
                           'id': 'query_notepad_ioc',
                           'link': 'https://devday2019.carbonblack.com/notepadReport',
                           'match_type': 'query',
                           'values': ['process_name:notepad.exe '
                                      'childproc_count:[1 TO *]']}],
              'link': 'https://devday2019.carbonblack.com/notep

## Ignoring reports or IOCs within Feeds
One more useful feature of the new model is that you don't have to use all reports or IOCs of the feed you subscribe to.
You can exclude specific parts from it. 
Note that, if report is part of the feed, we have to prefix it with "{feed_id}-" (where feed_id is actual feed_id)

Another note for CBR API users - in CBR you could not ignore individual IOCs, only reports.

In [37]:
ret = th.put_object(f"/threathunter/watchlistmgr/v3/orgs/{orgkey}/reports/{feed_id}-report3/iocs/query_notepad_ioc4/ignore", {})
pprint.pprint(ret.json())

{'ignored': True}


Or ignore entire report (all iocs)

In [38]:
ret = th.put_object(f"/threathunter/watchlistmgr/v3/orgs/{orgkey}/reports/{feed_id}-report3/ignore", {})
pprint.pprint(ret.json())

{'ignored': True}


In [39]:
ret = th.get_object(f"/threathunter/watchlistmgr/v3/orgs/{orgkey}/reports/{feed_id}-report3/ignore")
pprint.pprint(ret)

{'ignored': True}


## Running Watchlist on Historical Data
Once you have a watchlist set up (be it first creation, orafter  subsequent modification), you can execute this watchlist on the historical data. Here is example that executes historical evaluation of all reports in the watchlists watchlist for the last 7 days of data:

In [40]:
ret = th.post_object("/threathunter/search/v1/orgs/{orgkey}/processes/watchlist_evaluation", 
                     {
                         "watchlist_id": watchlist_id,
                         "cb.min_backend_timestamp": int(time.time()) - 86400*14,
                         "report_id": f"{feed_id}-report2"
                     })
pprint.pprint(ret)

<Response [204]>


Then we can use CBTH search API to validate that report hits have been found. For that, we will issue a search our feed-based report_id. Every matched process document will be taggeed with it.

In [45]:
ret = th.post_object("/threathunter/search/v1/orgs/{orgkey}/processes/search_jobs", {
                         "search_params": {
                             "q": f"report_id:{feed_id}-report2",
                             "sort": "backend_timestamp DESC"
                         }
                     })
query_id = ret.json()["query_id"]
time.sleep(2)
ret = th.get_object(f"/threathunter/search/v1/orgs/{orgkey}/processes/search_jobs/{query_id}/results?start_row=0&row_count=3")
pprint.pprint(ret)

{'data': [{'alert_id': ['7YZFGDDN-007fdb18-00000ad4-00000000-1d517750a2d6000-rjEcrqDUSyGwDRPt7A9Ocw-report2'],
           'backend_timestamp': '2019-06-03T18:41:40.947Z',
           'childproc_count': 2,
           'crossproc_count': 137,
           'device_id': 8379160,
           'device_name': 'enduser01',
           'device_timestamp': '2019-05-31T05:53:14.161Z',
           'filemod_count': 3,
           'index_class': 'default',
           'modload_count': 42,
           'netconn_count': 0,
           'org_id': '7YZFGDDN',
           'parent_guid': '7YZFGDDN-007fdb18-0000098c-00000000-1d51774fee39ac0',
           'parent_pid': 2444,
           'partition_id': 0,
           'process_guid': '7YZFGDDN-007fdb18-00000ad4-00000000-1d517750a2d6000',
           'process_hash': ['f2c7bb8acc97f92e987a2d4087d021b1',
                            '142e1d688ef0568370c37187fd9f2351d7ddeda574f8bfa9b0fa4ef42db85aa2'],
           'process_name': 'c:\\windows\\system32\\notepad.exe',
           'proc

## Report Search
ThreatHunter allows powerful way to search through your private as well as community reports

In [43]:
ret = th.get_object(f"/threathunter/feedsearch/v1/orgs/{orgkey}/search?query=notepad")
pprint.pprint(ret)

{'facets': {},
 'hits': {'hits': [{'_id': 'rjEcrqDUSyGwDRPt7A9Ocw-report3',
                    '_index': 'report_index-2018.11.17-1',
                    '_score': 15.579918,
                    '_source': {'access': 'private',
                                'description': 'Detected an instance of '
                                               'notepad.exe started by '
                                               'something else than '
                                               'explorer.exe',
                                'feed': {'feed_category': 'DevDay demo',
                                         'feed_id': 'rjEcrqDUSyGwDRPt7A9Ocw',
                                         'feed_name': 'Notepad attacks',
                                         'feed_provider_url': 'https://devday2019.carbonblack.com/notepadFeed',
                                         'feed_summary': 'Various attacks '
                                                         'using notepad'},
    

In [44]:
ret = th.get_object(f"/threathunter/feedsearch/v1/orgs/{orgkey}/search?query=notepad+AND+NOT+devday2019")
pprint.pprint(ret)

{'facets': {},
 'hits': {'hits': [{'_id': '1X7Zx5ZRbGc24xRp7IaQ-b3aefed8-e8b2-4671-8eb2-683f49ee687b',
                    '_index': 'report_index-2018.11.17-1',
                    '_score': 12.013574,
                    '_source': {'access': 'public',
                                'description': 'This query alerts on a process '
                                               'spawning both notepad and '
                                               'vssadmin. Ransomware uses '
                                               'vssadmin to delete all shadow '
                                               'copies, while notepad is used '
                                               'to display the ransom note.\n'
                                               '\n'
                                               'Threat:\n'
                                               'This behavior has been '
                                               'observed in Locky ransomware '
          

## Converting old CBR Query Reports

If you are converting old CBR query watchlist to new CBTH watchlist, you can use conversion API endpoint:

In [46]:
ret = th.post_object(f"/threathunter/feedmgr/v2/query/translate", 
                     {
                         "query":"digsig_publisher:microsoft OR (modload:kernel32.dll AND regmod:test123 AND original_filename:explorer.exe)"
                     })
pprint.pprint(ret.json())

{'query': '(process_publisher:microsoft OR (modload_name:kernel32.dll AND '
          'regmod_name:test123 AND process_original_filename:explorer.exe)) '
          '-legacy:true'}


## Watchlist telemetry
Here is how we can check watchlist telemetry. We can tell API how to aggregate our data (in minutes). As a result, we will get back number of **Executions** in a given time period, as well as number of **Hits** (matches wich convert to tags and/or alerts as configured in a given watchlist). This allows you to see how is given report performing and pinpoint any changes in hit rates. For example, you can see if last day has many more hits on average than the past week, which might indicate surge in specific type of attack within your org.

In [47]:
ret = th.get_object(f"/threathunter/watchlistmgr/v3/orgs/{orgkey}/watchlists/{watchlist_id}/telemetry?intervals=1440,60")
pprint.pprint(ret)

{'telemetry': [{'executions': 2,
                'hits': 0,
                'interval': 1440,
                'watchlist_id': 'sE6Hg7LRvqhy09kzxdxHw'},
               {'executions': 2,
                'hits': 0,
                'interval': 60,
                'watchlist_id': 'sE6Hg7LRvqhy09kzxdxHw'}]}


## Cleanup

First let's delete the feed we created. This will delete watchlist depending on it automatically

In [48]:
ret = th.get_object(f"/threathunter/feedsearch/v1/orgs/{orgkey}/search?query=DevDay")
for fid in set([r["_source"]["feed"]["feed_id"] for r in ret["hits"]["hits"]]):
    print(f"Deleting feed {fid}")
    th.delete_object("/threathunter/feedmgr/v2/orgs/7YZFGDDN/feeds/" + fid)



Deleting feed rjEcrqDUSyGwDRPt7A9Ocw


Finally, delete reports

In [49]:
ret = th.get_object(f"/threathunter/feedsearch/v1/orgs/{orgkey}/search?query=devday2019")
for rid in ([r["_id"] for r in ret["hits"]["hits"]]):
    print(f"Deleting report {rid}")
    try:
        th.delete_object(f"/threathunter/watchlistmgr/v3/orgs/{orgkey}/reports/{rid}")
    except Exception as e:
        print(f'...Failed to delete report {rid}: {e}')

Deleting report cqtxuHzRYeAlgn80AdFUA
Deleting report ROL9MGB7QKGbYcEk3wzmwg
Deleting report WORCC7gpRqC2LLBqbgup4A
